In [ ]:
    this is the first line of code inlind with text


    this is a line of code inline with text
LOL what was that singing

In [1]:
````
print('hello')
````

SyntaxError: invalid syntax (3336871262.py, line 1)

markdowwwwn


````
code block
````

In [11]:
import numpy as np
a = np.array([1,0,1])[:,None]
c = 3
print(a)

[[1]
 [0]
 [1]]


In [83]:
from collections import Counter

a = "The Dog jumps over the roof".lower()
b = "The cat jumps on the roof".lower()

vocab = Counter()
for word in a.split(' '):
    vocab[word] += 1
    
print(vocab)
print(len(vocab))

for i,word in enumerate(vocab):
    print(i,vocab[word])

Counter({'the': 2, 'dog': 1, 'jumps': 1, 'over': 1, 'roof': 1})
5
0 2
1 1
2 1
3 1
4 1


In [76]:
vocab = set()
vocab.add("ehllo")
vocab.add('ehllo')
vocab.add("Hello")
vocab.add("hello")
print(vocab)
for i,word in enumerate(vocab):
    print(i, word)
vocab_list = list(vocab)
print(vocab_list)
for i,word in enumerate(vocab_list):
    print(i, word)

{'Hello', 'hello', 'ehllo'}
0 Hello
1 hello
2 ehllo
['Hello', 'hello', 'ehllo']
0 Hello
1 hello
2 ehllo


In [79]:
init_weights = np.zeros([4,1])
print(init_weights)


[[0.]
 [0.]
 [0.]
 [0.]]


In [100]:
class Student:
    def __init__(self,age,name,update_age):
        self.age = age
        self.name = name
        self.update_age(update_age)
    def update_age(self, age):
        self.age = age
        print('Done! Your age is: ', self.age)
student1 = Student(18, 'eric', 19)

Done! Your age is:  19


In [87]:
assert(2==1)

AssertionError: 

In [150]:
import numpy as np

a = np.array([[1,2,3]])
b = np.array([[4],
              [5],
              [6]])
c = 10
print(b.dot(a))

[[ 4  8 12]
 [ 5 10 15]
 [ 6 12 18]]


In [123]:
a = np.random.rand(2,2)
b = np.zeros((2,2))
print(a.dot(10))
print(b)

[[7.70701954 3.82001083]
 [2.41216406 2.20468505]]
[[0. 0.]
 [0. 0.]]


In [135]:
a = "Cute dog"
if ("dog" in a.split(' ')):
    print("something")

something


In [147]:
zeros = np.zeros((1,10))
print(zeros)
print(zeros.size)
a = np.array([1,2,3,4])
print(a)
print(a.size)

zeros[8] += 1

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
10
[1 2 3 4]
4


IndexError: index 8 is out of bounds for axis 0 with size 1

In [155]:
a = np.array([1,2,3,4,5,6,7,8,9])
print(a[:-2])
print(a[-2:])
print('POSITIVE' == "POSITIVE")

[1 2 3 4 5 6 7]
[8 9]
True


In [156]:
a = [1]
a[9] = 10
print(a)

IndexError: list assignment index out of range

In [157]:
a = "german shepherd is the cutest"
for i,word in enumerate(a.split(' ')):
    print(i,word)

0 german
1 shepherd
2 is
3 the
4 cutest


In [176]:
a = np.random.rand(3,3)
print(a)
print(a[1][:])
print(a[1])

[[0.62735674 0.16241743 0.93646118]
 [0.11457933 0.6599085  0.98629201]
 [0.40312145 0.04266417 0.97324247]]
[0.11457933 0.6599085  0.98629201]
[0.11457933 0.6599085  0.98629201]


In [172]:
indices = np.zeros((1, 2))
print(indices)
indices[0][0] = 4
indices[0][1] = 10
print(indices)

for index in indices[0]:
    print(index)

[[0. 0.]]
[[ 4. 10.]]
4.0
10.0


In [177]:
a = "I like dogs and cats"
for i,word in enumerate(a.split(' ')):
    print(i, word)
    print(type(i))
    
    

0 I
<class 'int'>
1 like
<class 'int'>
2 dogs
<class 'int'>
3 and
<class 'int'>
4 cats
<class 'int'>


In [183]:
a = set([1,2,3])
print(a)
b = list(a)
print(b)
print(type(b[1]))

{1, 2, 3}
[1, 2, 3]
<class 'int'>


In [225]:
from collections import Counter
a = Counter()
review = "German shepherd is the cutest dog dog :D"
for word in review.split(' '):
    a[word] += 1
print(a)

word_count2 = Counter()

for word,cnt in list(a.most_common()):
    print(word)
    print(cnt)


Counter({'dog': 2, 'German': 1, 'shepherd': 1, 'is': 1, 'the': 1, 'cutest': 1, ':D': 1})
dog
2
German
1
shepherd
1
is
1
the
1
cutest
1
:D
1


In [187]:
a = np.array([[1,2,3]])
b = np.array(2)
print(a.dot(b))
print(b.dot(a))

[[2 4 6]]
[[2 4 6]]


In [210]:
if 1<2 and 2>1:
    print("correct")

correct


In [221]:
a = "dog cat"
if "dog" in a and 
    print(a)

dog cat


In [222]:
for i in range 5:
    print(i)

SyntaxError: invalid syntax (3929397374.py, line 1)